<a href="https://colab.research.google.com/github/riszwinger/tensorflow_cert/blob/main/basic/SentimentAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import os
import random
import shutil
import re
import string
print('GPU is ','avaliable :)' if tf.config.list_physical_devices('GPU') else ' not avaliable :(')
print("TF Version: {}".format(tf.__version__))

GPU is  avaliable :)
TF Version: 2.5.0


In [ ]:
url = "https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz"

data_dir=tf.keras.utils.get_file(fname='aclImdb_v1',origin=url,untar=True,)
print(data_dir)

/root/.keras/datasets/aclImdb_v1


In [ ]:
!ls /root/.keras/datasets/aclImdb


imdbEr.txt  imdb.vocab	README	test  train


In [ ]:
!ls /root/.keras/datasets/aclImdb/train


labeledBow.feat  pos	unsupBow.feat  urls_pos.txt
neg		 unsup	urls_neg.txt   urls_unsup.txt


In [ ]:
!ls /root/.keras/datasets/aclImdb/test


labeledBow.feat  neg  pos  urls_neg.txt  urls_pos.txt


## Need to  get rid of some uncesssary files

In [ ]:
train_dir="/root/.keras/datasets/aclImdb/train"
test_dir="/root/.keras/datasets/aclImdb/test"

In [ ]:
os.listdir(os.path.join(train_dir,'pos'))[:3]

['10217_9.txt', '2611_9.txt', '10105_8.txt']

## Random Data Check





In [ ]:
ip=random.choice(['pos','neg'])
fname=os.path.join(train_dir,ip,os.listdir(os.path.join(train_dir,ip))[random.randint(0,len(os.listdir(os.path.join(train_dir,ip))))])
print('File:',fname)
with open(fname) as f:
  print('Setiment: {}\nText: {}'.format(ip,f.read()))

File: /root/.keras/datasets/aclImdb/train/pos/7982_10.txt
Setiment: pos
Text: I am amazed that movies like this can still be made. I watch all kinds of movies all the time with my friends and i can say that this is one of the best i ever seen. Never thinked that a movie of 146 minutes can make me think about it on and on.<br /><br />Washington, charismatic and intense as ever, plays Creasy, a washed-up ex-counter-terrorist agent who's taken to the bottle. Once he's assigned to protect young Pita (Dakota Fanning) in Mexico City, his emotional and redemptive arc is jump-started in the way only an adorable little girl can provide. Inevitably, Pita is kidnapped by thugs, and Creasy decides that most of Mexico City must pay the price for daring to take away his character's teddy-bear-clutching catalyst. Yes, he has become...a Man on Fire. <br /><br />You must see this movie.


## Reading Data

In [ ]:
train_ds=tf.keras.preprocessing.text_dataset_from_directory(train_dir,seed=123,validation_split=0.2,subset='training',)
valid_ds=tf.keras.preprocessing.text_dataset_from_directory(train_dir,seed=123,validation_split=0.2,subset='validation')
test_ds=tf.keras.preprocessing.text_dataset_from_directory(test_dir,seed=123)


Found 75000 files belonging to 3 classes.
Using 60000 files for training.
Found 75000 files belonging to 3 classes.
Using 15000 files for validation.
Found 25000 files belonging to 2 classes.


## Something wrong with Training splits , as it reads 3 classes inteead of 2 so lets check

In [ ]:
train_ds.class_names

['neg', 'pos', 'unsup']

**It seems  it reads unsup folder as one of the class, so lets drop that folder**

In [ ]:
remove_dir = os.path.join(train_dir, 'unsup')
shutil.rmtree(remove_dir)

In [ ]:
train_ds=tf.keras.preprocessing.text_dataset_from_directory(train_dir,seed=123,validation_split=0.2,subset='training',)
valid_ds=tf.keras.preprocessing.text_dataset_from_directory(train_dir,seed=123,validation_split=0.2,subset='validation')
test_ds=tf.keras.preprocessing.text_dataset_from_directory(test_dir,seed=123)


Found 25000 files belonging to 2 classes.
Using 20000 files for training.
Found 25000 files belonging to 2 classes.
Using 5000 files for validation.
Found 25000 files belonging to 2 classes.


## Great , everything seems to be inorder

In [ ]:
for txt, lbl in train_ds.take(1):
  for i in range(3):
    print('Text: {}\nlabel: {} '.format(txt[i].numpy(),lbl[i].numpy()))

Text: b'After, I watched the films... I thought, "Why the heck was this film such a high success in the Korean Box Office?" Even thought the movie had a clever/unusal scenario, the acting wasn\'t that good and the characters weren\'t very interesting. For a Korean movie... I liked the fighting scenes. If you want to watch a film without thinking, this is the film for you. But I got to admit... the film was kind of childish... 6/10'
label: 1 
Text: b'Or released on DVD or screened on a cable channel like Amer. Life TV network. I have been watching another favorite, "Voyage to the Bottom of the Sea", as well as "Lost in Space" and Land of Giants". They\'ve been showing them forever but aren\'t receptive to suggestions for other shows. My father and I were big fans as I was already a big science/electronics nut, (still am) and my father was an old school chum of Nader. They both attended Oxy together. I still have memories of several of the episodes even though I was only 9. More so than 

In [ ]:
# Text: b"You're using the IMDb.<br /><br />You've given some hefty votes to some of your favourite films.<br /><br />It's something you enjoy doing.<br /><br />And it's all because of this. Fifty seconds. One world ends, another begins.<br /><br />How can it not be given a ten? I wonder at those who give this a seven or an eight... exactly how could THE FIRST FILM EVER MADE be better? For the record, the long, still opening shot is great showmanship, a superb innovation, perfectly suited to the situation. And the dog on the bike is a lovely touch. All this within fifty seconds.<br /><br />The word genius is often overused.<br /><br />THIS is genius."


## Standardization ---> Tokenization ---> Vectorization

In [ ]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [ ]:
re.escape(string.punctuation)

'!"\\#\\$%\\&\'\\(\\)\\*\\+,\\-\\./:;<=>\\?@\\[\\\\\\]\\^_`\\{\\|\\}\\~'

In [ ]:
'[%s]' % re.escape(string.punctuation)

'[!"\\#\\$%\\&\'\\(\\)\\*\\+,\\-\\./:;<=>\\?@\\[\\\\\\]\\^_`\\{\\|\\}\\~]'

In [ ]:
def custom_standarization(input_data):
  lowercase=tf.strings.lower(input_data)
  remove_brk=tf.strings.regex_replace(lowercase,'<br />',' ')
  remove_punc=tf.strings.regex_replace(remove_brk,'[%s]' % re.escape(string.punctuation),'')
  return remove_punc

In [ ]:
str1=b"You're using the IMDb.<br /><br />You've given some hefty votes to some of your favourite films.<br /><br />It's something you enjoy doing.<br /><br />And it's all because of this. Fifty seconds. One world ends, another begins.<br /><br />How can it not be given a ten? I wonder at those who give this a seven or an eight... exactly how could THE FIRST FILM EVER MADE be better? For the record, the long, still opening shot is great showmanship, a superb innovation, perfectly suited to the situation. And the dog on the bike is a lovely touch. All this within fifty seconds.<br /><br />The word genius is often overused.<br /><br />THIS is genius."
custom_standarization(str1).numpy()

b'youre using the imdb  youve given some hefty votes to some of your favourite films  its something you enjoy doing  and its all because of this fifty seconds one world ends another begins  how can it not be given a ten i wonder at those who give this a seven or an eight exactly how could the first film ever made be better for the record the long still opening shot is great showmanship a superb innovation perfectly suited to the situation and the dog on the bike is a lovely touch all this within fifty seconds  the word genius is often overused  this is genius'

In [ ]:
max_features=10000
max_seq=250
vectorize_layer=tf.keras.layers.experimental.preprocessing.TextVectorization(max_tokens=max_features,standardize=custom_standarization,output_sequence_length=max_seq)

In [ ]:
train_text=train_ds.map(lambda x,y:x)
vectorize_layer.adapt(train_text)

In [ ]:
def vetorize_text(input_data,label):
  text=tf.expand_dims(input_data,-1)
  return vectorize_layer(text),label

In [ ]:
text_val,label_val=next(iter(train_ds))

In [ ]:
first_text,first_lbl=text_val[0],label_val[0]
print('first_text:',custom_standarization(first_text.numpy()))
vec_txt,vec_lbl=vetorize_text(first_text,first_lbl)
print(vec_txt,'------',vec_lbl)

first_text: tf.Tensor(b'this documentary now available free on videogooglecom is a fantastic demonstration of the power of ordinary people to overcome injustice everyone must see this  chavez was elected in a landslide vote in 1998 his platform was to divert the fantastic oil wealth from the 20 middle class to the 80 poor he banned foreign drift net fishing in venezuelan waters he sent 10000 cuban doctors to the slums to treat the sick for free he wiped out illiteracy and set up new free universities   but it was his 30 tax on oil company profits that got him in trouble with the bush administration in 2002 while irish film makers kim bartley and donnacha obriain were interviewing chavez inside the presidential palace about his social programs a cia backed coup was launched with the cameras rolling chavez was captured and flown out of the country it was announced on national tv that he had resigned  but the poor of venezuela didnt believe the media they went to the palace in their milli

In [ ]:
vectorize_layer.get_vocabulary('of')

TypeError: ignored